In [1]:
import jiwer
import json


file_path = '/home/jinmyeong/code/hts/classifier/Data/audio_rationales_data.json'
with open(file_path, "r") as f:
    audio_rationales_data = json.load(f)
with open('/home/jinmyeong/code/hts/ASR/Data/ASR/whisperX_ASR_transcription_v4-HateSpeech-all-multispeaker.json', 'r') as fp:
    transcript_dict = json.load(fp)
with open('/home/jinmyeong/code/hts/classifier/Data/new_post_id_divisions.json', 'r') as fp:
    post_id_division = json.load(fp)

id_list = post_id_division['train'] + post_id_division['val'] + post_id_division['test']

transcript_rationales_dict = {}

for test_id in id_list:
    if audio_rationales_data.get(test_id):
        audio_datapoint = audio_rationales_data[test_id]

        audio_id = audio_datapoint['audio_id']
        gold_word_list = audio_datapoint['sentence']
        gold_text = " ".join(gold_word_list)
        gold_rationales = audio_datapoint['rationale']

        transcript = transcript_dict[test_id][0]['text'][1:]
        transcript_word_list = [word['word'] for word in transcript_dict[test_id][0]['words']]

        # transcript_rationales 0으로 초기화
        len_transcript_word = len(transcript_word_list)
        len_transcript_rationales = len(gold_rationales)

        transcript_rationales = [len_transcript_word * [0] for _ in range(len_transcript_rationales)]

        error = jiwer.process_words(gold_text, transcript)
        alignments = error.alignments[0]

        for alignment in alignments:
            align_type = alignment.type
            ref_start_idx = alignment.ref_start_idx
            ref_end_idx = alignment.ref_end_idx
            hyp_start_idx = alignment.hyp_start_idx
            hyp_end_idx = alignment.hyp_end_idx

            # align이 맞을 경우, ref의 rationale을 hyp에 넣는다.
            for idx, gold_rationale in enumerate(gold_rationales):
                if align_type=='equal':
                    transcript_rationales[idx][hyp_start_idx:hyp_end_idx] = gold_rationale[ref_start_idx:ref_end_idx]
                    
        transcript_rationales_dict[audio_id] = transcript_rationales

In [2]:
transcript_rationales_dict

{'1161241435313545217_twitter': [[0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1]],
 '21617110_gab': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1],
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1]],
 '1263614686156558336_twitter': [[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 '6465075_gab': [[0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0]],
 '1105628542023491584_twitter': [],
 '4428097_gab': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0